In [44]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from nltk.corpus import stopwords
#import nltk
#nltk.download('stopwords')

import os

In [45]:
file_names = []

if os.path.exists('../data/') and os.path.isdir('../data/'):
    file_names = os.listdir('../data/')

file_names = ['../data/'+file_name for file_name in file_names]

file_names

['../data/codigo_penal.txt',
 '../data/codigo_de_etica_publica.txt',
 '../data/codigo_procesal_penal_federal.txt',
 '../data/codigo_aeronautico.txt',
 '../data/codigo_procesal_civil_y_comercial.txt',
 '../data/codigo_aduanero.txt',
 '../data/constitucion-caba.txt',
 '../data/codigo_electoral_nacional.txt',
 '../data/codigo_civil_y_comercial.txt',
 '../data/codigo_alimentario_argentino.txt',
 '../data/constitucion-nacional.txt',
 '../data/codigo_de_mineria.txt']

In [46]:
stop_words = set(stopwords.words('spanish'))

def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

# apply stop-word removal on your dataset before tokenizing
processed_files = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        text = file.read()
        processed_text = remove_stop_words(text)
        processed_files.append(processed_text)

In [49]:
processed_files[0].split()

['LEY',
 'N°',
 '11.179',
 'CODIGO',
 'PENAL',
 'NACION',
 'Buenos',
 'Aires,',
 '29',
 'octubre',
 '1921',
 'Senado',
 'Cámara',
 'Diputados',
 'Nación',
 'Argentina,',
 'reunidos',
 'Congreso,',
 'etc.,',
 'sancionan',
 'fuerza',
 'LEY:',
 'LIBRO',
 'PRIMERO',
 'DISPOSICIONES',
 'GENERALES',
 'TITULO',
 'I',
 'Aplicación',
 'ley',
 'penal',
 'Artículo',
 '1.°',
 '-',
 'código',
 'aplicará:',
 '1.º',
 'delitos',
 'cometidos',
 'cuyos',
 'efectos',
 'deban',
 'producirse',
 'territorio',
 'Nación',
 'Argentina,',
 'lugares',
 'sometidos',
 'jurisdicción;',
 '2.º',
 'delitos',
 'cometidos',
 'extranjero',
 'agentes',
 'empleados',
 'autoridades',
 'argentinas',
 'desempeño',
 'cargo.',
 'Art.',
 '2.°',
 '-',
 'Si',
 'ley',
 'vigente',
 'tiempo',
 'cometerse',
 'delito',
 'fuere',
 'distinta',
 'exista',
 'pronunciarse',
 'fallo',
 'tiempo',
 'intermedio,',
 'aplicará',
 'siempre',
 'benigna.',
 'Si',
 'condena',
 'dictare',
 'ley',
 'benigna,',
 'pena',
 'limitará',
 'establecida',
 'le

In [37]:
tokenizer = ByteLevelBPETokenizer()

# ensure tokenizer reserves the special tokens for the vocab
# (using the same as RoBERTa needs)
special_tokens = [
    "<pad>",
    "<unk>",
    "<s>",
    "</s>",
    "<mask>"
]

tokenizer.train_from_iterator(processed_files,
                              vocab_size=30000,
                              min_frequency=2,
                              special_tokens=special_tokens
                              )

Save files to disk

In [38]:
token_dir = '../LawBERTarg/'
if not os.path.exists(token_dir):
    os.makedirs(token_dir)
tokenizer.save_model(token_dir)

['../LawBERTarg/vocab.json', '../LawBERTarg/merges.txt']

## Testing tokenizer

In [39]:
tokenizer = ByteLevelBPETokenizer(vocab = '../LawBERTarg/vocab.json',
                                  merges = '../LawBERTarg/merges.txt',
                                  lowercase = False,)

Example encodings:

In [40]:
print(tokenizer.encode("la ley prohibe a").tokens)

['la', 'Ġley', 'Ġprohib', 'e', 'Ġa']


In [41]:
print(tokenizer.encode("Aquel que dañe obra pública u obrare en").tokens)

['A', 'quel', 'Ġque', 'ĠdaÃ±', 'e', 'Ġobra', 'ĠpÃºblica', 'Ġu', 'Ġobrare', 'Ġen']


Add start and end tokens:

In [42]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)

In [43]:
print(tokenizer.encode("legalidad, gestión y auditoría de toda la actividad de la administración pública centralizada y descentralizada").tokens)

['<s>', 'leg', 'alidad', ',', 'ĠgestiÃ³n', 'Ġy', 'ĠauditorÃŃa', 'Ġde', 'Ġtoda', 'Ġla', 'Ġactividad', 'Ġde', 'Ġla', 'ĠadministraciÃ³n', 'ĠpÃºblica', 'Ġcentralizada', 'Ġy', 'Ġdescentralizada', '</s>']
